<a href="https://colab.research.google.com/github/ranjanguddu/ENDPhase1/blob/main/%20ENDPhase1/Session-4/session-4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [5]:
for i in range(10):
  print(vars(train_data.examples[i]).get('text'))


['A', 'Brother', "'s", 'Promise', 'is', 'a', 'wonderful', 'family', 'film', '.', 'This', 'is', 'a', 'biography', 'of', 'Dan', 'Jansen', ',', 'a', 'champion', 'Olympic', 'speed', 'skater', '.', 'The', 'movie', 'depicts', 'this', 'athlete', "'s", 'life', 'from', 'a', 'young', 'age', 'through', 'full', 'adulthood', '.', 'The', 'love', 'and', 'support', 'of', 'the', 'family', 'members', 'is', 'evident', 'throughout', '.', 'How', 'Dan', 'and', 'the', 'rest', 'of', 'his', 'family', 'handle', 'winning', 'and', 'losing', 'races', 'is', 'a', 'life', 'lesson', 'for', 'all', 'of', 'us', '.', 'The', 'commitment', 'and', 'determination', 'of', 'Dan', "'s", 'coach', 'and', 'his', 'teammates', ',', 'shows', 'what', 'it', 'takes', 'to', 'make', 'a', 'real', 'team', '.', 'How', 'Dan', 'and', 'his', 'family', 'deal', 'with', 'a', 'devastating', 'illness', 'of', 'a', 'loved', 'one', 'is', 'depicted', 'without', 'undo', 'sentiment', 'or', 'sugarcoating', '.', 'The', 'faith', 'of', 'the', 'family', 'is', '

In [6]:
for i in range(len(train_data)):
  (vars(train_data.examples[i]).get('text').reverse())

In [7]:
for i in range(10):
  print(vars(train_data.examples[i]).get('text'))


['.', 'age', 'any', 'of', 'person', 'a', 'for', 'meaningful', 'be', 'can', 'which', 'film', 'family', 'powerful', 'a', 'is', 'This', '.', 'lives', 'their', 'of', 'part', 'major', 'a', 'obviously', 'is', 'and', 'terms', 'basic', 'in', 'shown', 'is', 'family', 'the', 'of', 'faith', 'The', '.', 'sugarcoating', 'or', 'sentiment', 'undo', 'without', 'depicted', 'is', 'one', 'loved', 'a', 'of', 'illness', 'devastating', 'a', 'with', 'deal', 'family', 'his', 'and', 'Dan', 'How', '.', 'team', 'real', 'a', 'make', 'to', 'takes', 'it', 'what', 'shows', ',', 'teammates', 'his', 'and', 'coach', "'s", 'Dan', 'of', 'determination', 'and', 'commitment', 'The', '.', 'us', 'of', 'all', 'for', 'lesson', 'life', 'a', 'is', 'races', 'losing', 'and', 'winning', 'handle', 'family', 'his', 'of', 'rest', 'the', 'and', 'Dan', 'How', '.', 'throughout', 'evident', 'is', 'members', 'family', 'the', 'of', 'support', 'and', 'love', 'The', '.', 'adulthood', 'full', 'through', 'age', 'young', 'a', 'from', 'life', "'s

In [8]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [9]:

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)


In [10]:

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [11]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
       
        #text = [sent len, batch size]
        #print(f'text:{text}')
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu())
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [12]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,387,817 trainable parameters


In [14]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [15]:
model.embedding.weight.data.copy_(pretrained_embeddings)


tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])

In [16]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])


In [17]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [18]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [19]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        #print(f'text here:{text}')

        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [23]:

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 41s
	Train Loss: 0.672 | Train Acc: 57.76%
	 Val. Loss: 0.552 |  Val. Acc: 72.39%
Epoch: 02 | Epoch Time: 2m 43s
	Train Loss: 0.510 | Train Acc: 75.30%
	 Val. Loss: 0.398 |  Val. Acc: 82.61%
Epoch: 03 | Epoch Time: 2m 43s
	Train Loss: 0.391 | Train Acc: 82.69%
	 Val. Loss: 0.387 |  Val. Acc: 84.46%
Epoch: 04 | Epoch Time: 2m 43s
	Train Loss: 0.315 | Train Acc: 87.16%
	 Val. Loss: 0.387 |  Val. Acc: 84.83%
Epoch: 05 | Epoch Time: 2m 43s
	Train Loss: 0.261 | Train Acc: 89.85%
	 Val. Loss: 0.310 |  Val. Acc: 87.42%
Epoch: 06 | Epoch Time: 2m 43s
	Train Loss: 0.223 | Train Acc: 91.31%
	 Val. Loss: 0.311 |  Val. Acc: 87.41%
Epoch: 07 | Epoch Time: 2m 44s
	Train Loss: 0.180 | Train Acc: 93.16%
	 Val. Loss: 0.337 |  Val. Acc: 86.60%
Epoch: 08 | Epoch Time: 2m 44s
	Train Loss: 0.153 | Train Acc: 94.27%
	 Val. Loss: 0.414 |  Val. Acc: 85.73%
Epoch: 09 | Epoch Time: 2m 43s
	Train Loss: 0.139 | Train Acc: 94.95%
	 Val. Loss: 0.318 |  Val. Acc: 88.24%
Epoch: 10 | Epoch T

In [24]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.371 | Test Acc: 84.22%


In [25]:

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 44s
	Train Loss: 0.243 | Train Acc: 90.50%
	 Val. Loss: 0.315 |  Val. Acc: 87.00%
Epoch: 02 | Epoch Time: 2m 44s
	Train Loss: 0.195 | Train Acc: 92.88%
	 Val. Loss: 0.305 |  Val. Acc: 88.27%
Epoch: 03 | Epoch Time: 2m 43s
	Train Loss: 0.168 | Train Acc: 93.91%
	 Val. Loss: 0.319 |  Val. Acc: 88.24%
Epoch: 04 | Epoch Time: 2m 43s
	Train Loss: 0.145 | Train Acc: 94.76%
	 Val. Loss: 0.325 |  Val. Acc: 88.85%
Epoch: 05 | Epoch Time: 2m 43s
	Train Loss: 0.139 | Train Acc: 95.01%
	 Val. Loss: 0.343 |  Val. Acc: 88.14%
Epoch: 06 | Epoch Time: 2m 43s
	Train Loss: 0.113 | Train Acc: 95.93%
	 Val. Loss: 0.371 |  Val. Acc: 88.09%
Epoch: 07 | Epoch Time: 2m 42s
	Train Loss: 0.095 | Train Acc: 96.65%
	 Val. Loss: 0.360 |  Val. Acc: 88.59%
Epoch: 08 | Epoch Time: 2m 43s
	Train Loss: 0.080 | Train Acc: 97.31%
	 Val. Loss: 0.416 |  Val. Acc: 88.40%
Epoch: 09 | Epoch Time: 2m 42s
	Train Loss: 0.070 | Train Acc: 97.64%
	 Val. Loss: 0.419 |  Val. Acc: 88.69%
Epoch: 10 | Epoch T

In [26]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.357 | Test Acc: 86.25%
